In [ ]:
!pip install -U spacy
!pip install jsonlines

     |████████████████████████████████| 6.2 MB 5.2 MB/s 
     |████████████████████████████████| 653 kB 48.4 MB/s 
     |████████████████████████████████| 10.1 MB 38.0 MB/s 
     |████████████████████████████████| 181 kB 63.7 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
     |████████████████████████████████| 457 kB 61.2 MB/s 
     |████████████████████████████████| 58 kB 6.4 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: smart-open
    Found existing installation: smart-open 6.0.0
    U

In [ ]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 400.7 MB 6.0 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


# **Imports**

In [ ]:
import spacy
from spacy.pipeline import EntityRuler
from spacy.lang.en import English
from spacy.tokens import Doc

In [ ]:
#gensim
import gensim
from gensim import corpora

In [ ]:
#Data loading/ Data manipulation
import pandas as pd
import numpy as np
import jsonlines

In [ ]:
#nltk
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download(['stopwords','wordnet'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
#warning
import warnings 
warnings.filterwarnings('ignore')

## **Loading Spacy Model**

In [ ]:
nlp = spacy.load("en_core_web_lg")
skill_pattern_path = "./jz_skill_patterns.jsonl"

# **Entity Ruler**


In [ ]:
ruler = nlp.add_pipe("entity_ruler")
ruler.from_disk(skill_pattern_path)
nlp.pipe_names

['tok2vec',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ner',
 'entity_ruler']

In [ ]:
def unique_skills(x):
    return list(set(x))

def get_skills(text):
    doc = nlp(text)
    myset = []
    subset = []
    for ent in doc.ents:
        if ent.label_ == "SKILL":
            subset.append(ent.text)
    myset.append(subset)
    return unique_skills(subset)

In [ ]:
data = pd.read_csv("Resume.csv")
data

,id,category,resume
0,1,software developer,SOFTWARE DEVELOPER INTERN ...
1,2,software developer,SOFTWARE DEVELOPER Professional...
2,3,software developer,GIS ANALYST/SOFTWARE DEVELOPER ...
3,4,software developer,SENIOR SOFTWARE DEVELOPER Caree...
4,5,software developer,SENIOR SOFTWARE DEVELOPER Summa...
...,...,...,...
436,545,java developer,GRADUATE RESEARCH ASSISTANT Pro...
437,546,java developer,GRADUATE RESEARCH ASSISTANT ...
438,547,java developer,STUDENT ASSISTANT Qualificati...
439,548,java developer,YW TEAM LEAD Summary ...


In [ ]:
def clean_resumes(data):
    review = re.sub(
        '(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?"',
        " ",
        data,
    )
    review = review.lower()
    review = review.split()
    lm = WordNetLemmatizer()
    review = [
        lm.lemmatize(word)
        for word in review
        if not word in set(stopwords.words("english"))
    ]
    review = " ".join(review)
    return review

In [ ]:
#cleaning data
data['cleaned_resume'] = data.resume.apply(lambda x: clean_resumes(x))

data['skills'] = data.cleaned_resume.apply(lambda x: get_skills(x))
data

,id,category,resume,cleaned_resume,skills
0,1,software developer,SOFTWARE DEVELOPER INTERN ...,software developer intern summary skill profic...,"[buddy, linux, commerce, analytics, embedded s..."
1,2,software developer,SOFTWARE DEVELOPER Professional...,software developer professional summary obtain...,"[nginx, linux, code review, issue tracking, ja..."
2,3,software developer,GIS ANALYST/SOFTWARE DEVELOPER ...,gi analyst software developer summary contribu...,"[spatial analysis, framework, marketing, ruby,..."
3,4,software developer,SENIOR SOFTWARE DEVELOPER Caree...,senior software developer career focus work en...,"[commerce, java, junit, framework, code covera..."
4,5,software developer,SENIOR SOFTWARE DEVELOPER Summa...,senior software developer summary oracle certi...,"[linux, authentication, java, framework, junit..."
...,...,...,...,...,...
436,545,java developer,GRADUATE RESEARCH ASSISTANT Pro...,graduate research assistant professional profi...,"[remote control, analytics, java, framework, d..."
437,546,java developer,GRADUATE RESEARCH ASSISTANT ...,graduate research assistant summary obtain int...,"[cryptographic protocol, linux, authentication..."
438,547,java developer,STUDENT ASSISTANT Qualificati...,student assistant qualification algorithm java...,"[linux, authentication, android studio, java, ..."
439,548,java developer,YW TEAM LEAD Summary ...,yw team lead summary motivated team lead posse...,"[linux, code review, java, react, support, sof..."


In [ ]:
data.to_csv('./resume_with_skills.csv')